## Outline 

* Neural Net Traning Workflow

* PyTorch Data Type: Tensors

* Graph Computation and Neural Net Models

* Example: Iris Dataset Classification

* Assignment: MNIST Classification

### Part 1: Neural Net Training Workflow
1. Prepare Data
    * Define batch size
    * Split train/val/test sets
    * Migrate to Tensors
    * Additional pre-processing (normalization, one hot encoding, etc.)

    One hot encoding: Transforms categorical data into one hot vectors. 1 in the index representing the class, 0 in all other indices. 
    
    Use *torch.nn.functional.one_hot()* 

2. Select Hyperparameter
    
    * Network size and type
    * Learning Rate
    * Regularizers and strength
    * Loss function and optimizer
    * Other hyperparameters

3. Define Model 
    * Network type
    * Network parameters/layers
    * Output values(s) and dimensions
    * Forward() Function

4. Identiify Tracked Values

    * Traning Loss
    * Validation Loss
    * Other relevant values

5. Train and Validate Model

    * Train Model: 
        * Calculate loss on training set
        * Backpropagation gradients
        * Update weights
    * Validate Model:
        * Calculate error on validation or test set
        * Do not update weights
    * Save losses in placeholders

6. Visulization and Evaluation
    * Visualize Traning Progress: Convergence, over or under fitting
    * Evaluate model: Confusion matrix, generate samples, identify model weakness




### Part 2: Tensors

* Main data structure for PyTorch
* Like numpy arrays, but optimized for machine learning 
    * Can be migrated to or stored on GPUs
    * Optimized for automatic differentiation
* Three main attributes:
    * Shape - size of each dimension
    * Datatype - form of each entry (float, int, etc.)
    * Device - cpu or cuda (gpu)

**Tensor Initialization**

Can create tensor from existing data: *torch.Tensor([[1,2],[3,4]])*, *torch.Tensor(np_array)*

Can generate tensor with random or fixed values: *torch.ones(shape)*, *torch.rand(shape)*

**Tensor Operations**

<img src="images/2_3.png" width ="500" height="300" alt="centered image" />


### Part 3: Graph Computation and Neural Net Models

**Bass Class**: nn.Module

**Two primary features of base class**: Parameters, Forward function

**Common PyTorch Layers**:
* Linear
* Activation Functions (ReLU, tanj, etc.)
* Dropout
* RNN
* Convolution

**Neural Network Models**

<img src="images/2_4.png" width ="700" height="400" alt="centered image" />


### Computational graphs
* PyTorch generates a computational graph every time a parameter or variable with requires_grad is operated on
* The graph is used to back-propagate errors and update the parameters

### Loss functions and optimizers
* **Loss function**: example nn.MSELoss() for regression, nn.NLLLoss() or nn.CrossEntropy() for classification
* **Optimizer**: example optim.SGD, optim.Adam


```
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
```

## Optimization during training

* Each traning step, the optimization occurs in 3 steps
    * optimizer.zero_grad() -Resets the accumulated gradients
    * loss.backward() -Back-propagates the gradients to assign the contribution from each parameter
    * optimizer.step() -Updates the parameters based on the gradients, according to the optimization scheme (optimizer)
**Saving and Loading Models**
* Userful quantities to track during traning: training loss, validation loss, model state dictionary (parameters), optimizer state dictionary
* Loading state dictionaries: model.load_statedict() loads the saved parameter weights into the model, optimizer.load_statedict() loads optimizer state, such as learning rate, momentum, etc.

* Can use torch.save() and torch.load() 
* Create checkpoints to save all in one file
* Can save every eopch, or define some condition for saving (best loss, every n eopchs, etc.)

## Example: Iris Dataset Classification


```Python
from skelarn.datasets import load_iris
from sklearn.model_selection import tranin_test_split
from sklearn.preprocessing import StandardScaler

iris = load_iris()
X = iris['data']
y = iris['target']
names = iris['target_names']
feature_names = iris['feature_names']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=2
)

```









Dataset containing characteristics of 3 different flower types. Found in sklearn.datasets. 

* 3-layer fully connected neural net
* 1^{st} and 2^{nd} layer have 50 neurons each
* Final layer has 3 neurons for classification

* Activation functions can also be called using torch.nn.functional



**Model Definition**

```Python
import torch.nn.functional as F
class Model(nn.Module):
    def _init_(self,input_dim):

        super(Model,self)._init_()
        self.layer1 = nn.Linear(input_dim,50)
        self.layer2 = nn.Linear(50,50)
        self.layer3 = nn.Linear(50,3)

    def forward(self,x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        x = F.softmax(self.layer3(x), dim=1)
        return x

```

**Data, hyperparameters, and Saved Values**
* Define loss and optimizer

* Define training eopchs and data
* Tracking loss and accuracy at each epoch

```Python
import tqdm

EPOCHS = 100
X_train = torch.from_numpy(X_train).float()
y_train = torch.from_numpy(y_train).float()
X_test = torch.from_numpy(X_test).float()
y_test = torch.from_numpy(y_test).float()

loss_list = np.zeros((EPOCHS,))
accuracy_list = np.zeros(EPOCHS,))

model = MOdel(X_train.shape[1])
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()

```

**Model Traning and validation**

* Output of model gives predictions

* Track traning loss as loss

* Check accuracy on validation set

```Python
for epoch in tqdm.trange(EPOCHS):
    y_pred = model(X_train)
    loss = loss_fn(y_pred, y_train)
    loss_list[epoch] = loss.item()

    # Zero gradients
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    with torch.no_grad():
        y_pred = model(X_test)
        correct = (torch.argmax(y_pred, dim=1) == y_test.type(torch.FloatTensor)
        accuracy_list[epoch] = correct.mean()
```

**Plot Validation accuracy and training loss**

```Python
fig, (ax1, ax2) = plt.subplots(2, figsize=(12, 6), sharex=True)

ax1.plot(accuracy_list)
ax1.set_ylabel('validation accuracy')
ax2.plot(loss_list)
ax2.set_ylabel('training loss')
ax2.set_xlabel('eopchs')

```